In [8]:
import joblib
import pandas as pd
import random
import json
import re
import spacy
nlp = spacy.load("en_core_web_sm")
import string
import os

In [2]:
# 加载.joblib文件
old_data = joblib.load('/home/gy237/project/Biomedical_datasets/total_pubmed/abstracts_1-1252.joblib')
print(old_data.columns)
# ['pmid', 'title', 'abstract', 'journal', 'pubdate', 'authors','mesh_terms']

Index(['pmid', 'title', 'abstract', 'journal', 'pubdate', 'authors',
       'mesh_terms', 'pub_year', 'pub_month'],
      dtype='object')


In [3]:
new_data = joblib.load('/home/gy237/project/Biomedical_datasets/total_pubmed/abstracts_1253-1575.joblib')
print(new_data.columns)

Index(['pmid', 'title', 'abstract', 'journal', 'authors', 'mesh_terms',
       'pub_year', 'pub_month'],
      dtype='object')


In [4]:
combined_df = pd.concat([old_data, new_data], axis=0)
print('去重前：', len(combined_df))
data = combined_df.drop_duplicates(subset=['pmid'], keep='last')
print('去重后：', len(data))
print(data.columns)

去重前： 39390916
去重后： 37814430
Index(['pmid', 'title', 'abstract', 'journal', 'pubdate', 'authors',
       'mesh_terms', 'pub_year', 'pub_month'],
      dtype='object')


In [5]:
# 以data作为keyword进行筛选abstract
print('data',len(data))
filtered_df = data[data['abstract'].str.contains(' data', case=False)]
print('filtered_df', len(filtered_df))

data 37814430
filtered_df 5446108


In [16]:
# 将abstracts分句并进行保存
def ends_with_punctuation(text):
    return text[-1] in string.punctuation if text else False

def split_abstracts(abstract):
    doc = nlp(abstract)
    abstracts = [sent.text for sent in doc.sents]
    assert len(abstracts) != 0
        
    new_abstracts = []
    for j in abstracts:
        j = j.strip().split('\n')   # 有些内部会有\n
        for k in j:
            if len(k.split(' ')) > 5 and ends_with_punctuation(k):
                new_abstracts.append(k)
    return new_abstracts

def save_abstracts(data, name):
    for index, row in data.iterrows():
        pmid = row['pmid']
        abstract = row['abstract']
        doc = split_abstracts(abstract)
        
        # 以pmid为文件名创建txt文件
        with open(f"/home/gy237/project/Biomedical_datasets/total_pubmed/{name}/{pmid}.txt", "w", encoding="utf-8") as file:
            for j in doc:
                file.write(j + '\n')


In [17]:
f1 = '/home/gy237/project/Biomedical_datasets/total_pubmed/Batch_1/agreement'
f2 = '/home/gy237/project/Biomedical_datasets/total_pubmed/Batch_2/Batch_2'
exist_pmids = os.listdir(f1) + os.listdir(f2)
exist_pmids = [i.split('.')[0] for i in exist_pmids]
print(len(exist_pmids))
filtered_df_ = filtered_df[~filtered_df['pmid'].isin(exist_pmids)]
print(len(filtered_df_))


batch = filtered_df_.sample(n=100, random_state=42)
save_abstracts(batch, 'Batch_3')

200
5445924


In [4]:
pubdata = joblib.load('/home/gy237/project/Biomedical_datasets/total_pubmed/New_PMID_abstract/abstracts_and_pubdata/pubdate_1253-1575.joblib')
print(pubdata.columns)

Index(['pmid', 'pub_year', 'pub_month'], dtype='object')


In [6]:
# 过滤出 'column_name' 为空的行
count = []
null_rows = pubdata[pubdata['pub_year'].isnull()]
print(null_rows.shape)
for i in pubdata['pub_year']:
    if len(i) != 4:
        count.append(i)
print(count)
print(len(count))

(0, 3)
['Spring', 'Spring', 'Winter', 'Winter', 'Winter', 'Spring', 'Autumn', 'Spring', 'Summer', 'Winter', 'Summer', 'Winter', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Summer', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Winter', 'Spring', 'Summer', 'Winter', 'Summer', 'Summer', 'Summer', '2005-2006', '2007-2008', 'Summer', 'Winter', 'Summer', 'Summer', 'Spring', 'Spring', 'Winter', 'Winter', 'Winter', 'Spring', 'Spring', 'Winter', 'Summer', '1992-1993', 'Spring']
98


In [4]:
# 通过布尔索引筛选出 pmid 为指定值的行
filtered_data = data[data['pmid'] == '21370410']

# 打印筛选出的条目
filtered_data.to_json('/home/gy237/project/Biomedical_datasets/total_pubmed/New_PMID_abstract/test_.json', orient='records', lines=True)

In [30]:
# 保留头尾5行
data.tail().to_json('/home/gy237/project/Biomedical_datasets/total_pubmed/tail5_data.json', orient='records', lines=True)
data.head().to_json('/home/gy237/project/Biomedical_datasets/total_pubmed/top5_data.json', orient='records', lines=True)

In [26]:
# 从文件中抽取所有的PMID
pmid_list = data['pmid'].tolist()
print(len(pmid_list))
with open('/home/gy237/project/Biomedical_datasets/total_pubmed/pmid_list.txt', 'w') as file:
    for i in pmid_list:
        file.write(i + '\n')

36502590


# To get new total pmids
export PATH=${HOME}/edirect:${PATH}
esearch -db pubmed -query "2023/10/01:2024/9/10[dp]" | efetch -format uid > /Users/gy237/Desktop/python/pmids.txt
2023/01/01:2024/9/10[dp]
2023/10/01:2024/9/10[dp]

In [5]:
# Merge new and old pmid to get total PMID
with open('/home/gy237/project/Biomedical_datasets/total_pubmed/PMID_1614421.txt', 'r') as file:
    new = file.readlines()
new = [i.strip() for i in new]
with open('/home/gy237/project/Biomedical_datasets/total_pubmed/PMID_36502590.txt', 'r') as file:
    old = file.readlines()
old = [i.strip() for i in old]
print('New', len(new))
print('Old', len(old))

total = old + new
total = list(set(total))
print('Total', len(total))
with open('/home/gy237/project/Biomedical_datasets/total_pubmed/PMID_37676329.txt', 'w') as file:
    for i in total:
        file.write(i + '\n')


1614421
36502590
37676329


In [5]:
# Get unique new PMIDs
with open('/home/gy237/project/Biomedical_datasets/total_pubmed/PMID_1614421.txt', 'r') as file:
    new = file.readlines()
new = [i.strip() for i in new]
with open('/home/gy237/project/Biomedical_datasets/total_pubmed/PMID_36502590.txt', 'r') as file:
    old = file.readlines()
old = [i.strip() for i in old]
print('New', len(new))
print('old', len(old))

# 将两个list转换为集合
set1 = set(new)
set2 = set(old)
print('New', len(set1))
print('old', len(set2))

# 找出list1中有而list2中没有的元素
result = set1 - set2

# 如果需要将结果转换回list
result_list = list(result)

print(len(result_list))
with open('/home/gy237/project/Biomedical_datasets/total_pubmed/PMID_unique_new.txt', 'w') as file:
    for i in result_list:
        file.write(i + '\n')

New 1614421
old 36502590
New 1614421
old 36502590
1173739


In [5]:
# 随机抽取200条PMID
with open('/home/gy237/project/Biomedical_datasets/total_pubmed/PMID_37676329.txt', 'r') as file:
    total = file.readlines()
total = [i.strip() for i in total]
random.seed(42)
sample_200 = random.sample(total, 300)
with open('/home/gy237/project/Biomedical_datasets/total_pubmed/sample_200/sample_300.txt', 'w') as file:
    for i in sample_200:
        file.write(i + '\n')

In [7]:
# 从全部数据中匹配抽出来的pmid的abstract
with open('/home/gy237/project/Biomedical_datasets/total_pubmed/sample_200/sample_300.txt', 'r') as file:
    sample_200 = file.readlines()
sample_200 = [i.strip() for i in sample_200]
print(len(sample_200))

dic_list = []
empty = []
for pmid in sample_200:
    # 获取特定PMID的行
    row = data[data['pmid'] == pmid]
    if not row.empty:
        row_dict = row.to_dict(orient='records')[0]
        dic_list.append(row_dict)
        print(len(dic_list), end=' ')
    else:
        empty.append(pmid)
# 将字典保存为JSON文件
with open(f'/home/gy237/project/Biomedical_datasets/total_pubmed/sample_200/sample_300_abstract.json', 'w', encoding='utf-8') as f:
    json.dump(dic_list, f, ensure_ascii=False, indent=4)


300
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 

In [9]:
print(empty)
print(len(dic_list))
with open('/home/gy237/project/Biomedical_datasets/total_pubmed/sample_200/empty.txt', 'w') as file:
    for i in empty:
        file.write(i + '\n')

['38174763', '38168420', '38240506', '39066312', '39133620', '38319808', '39076848', '38769345']
292


# To get abstract
export PATH=${HOME}/edirect:${PATH}
cat /Users/gy237/Desktop/python/PMID_unique_new.txt | efetch -db pubmed -format medline > /Users/gy237/Desktop/python/PMID_unique_new_abstracts.txt

PMID- pmid
TI  - title
AB  - abstract
JT  - journal
DP  - pubdate
FAU - authors        多行
MH  - mesh_terms     多行

In [10]:
def get_authors(data):
    # 抽取authors
    authors_list = []
    current_author = {}

    # 按行读取数据
    for line in data.strip().splitlines():
        if line.startswith("FAU - "):
            # 提取姓氏和名字
            fau_match = re.search(r"FAU - ([^,]+), (.+)", line)
            if fau_match:
                current_author['last_name'] = fau_match.group(1)
                current_author['first_name'] = fau_match.group(2)
        
        elif line.startswith("AU  - "):
            # 提取缩写
            au_match = re.search(r"AU  - (.+)", line)
            if au_match:
                current_author['initials'] = au_match.group(1).split()[-1]
        
            
                # 将当前作者添加到列表
            authors_list.append(f"{current_author['last_name']}|{current_author['first_name']}|{current_author['initials']}|")
            current_author = {}

    # 生成最终的字符串
    authors_str = ";".join(authors_list)
    return authors_str

In [11]:
def extract_dic(txt):
    pmid = txt.split('\n')[0]
    txt = txt.replace('\n      ', '')
    authors = get_authors(txt)

    mesh_terms_list = []
    for line in txt.strip().splitlines():
        if line.startswith("TI  - "):
            title = line.split('TI  - ')[1].split('\n')[0]
        
        elif line.startswith("AB  - "):
            abstract = line.split('AB  - ')[1].split('\n')[0]
        
        elif line.startswith("JT  - "):
            journal = txt.split('JT  - ')[1].split('\n')[0]
        
        elif line.startswith("DP  - "):
            pubdate = line.split('DP  - ')[1].split('\n')[0]

        elif line.startswith("MH  - "):
            mesh_terms = line.split('MH  - ')[1].split('\n')[0]
            mesh_terms_list.append(mesh_terms)
    mesh_terms = '; '.join(mesh_terms_list)
        
    dic = {'pmid': pmid, 'title':title, 'abstract':abstract, 'journal': journal, 'pubdate': pubdate, 'authors':authors, 'mesh_terms': mesh_terms}

    return pmid, dic

In [12]:
# 抽取新下载的数据的abstract等metadata
with open('/home/gy237/project/Biomedical_datasets/total_pubmed/sample_200/sample_8_abstracts.txt', 'r', encoding='utf-8') as file:
    text = file.read()
text_list = text.split('\nPMID- ')
text_list = [i for i in text_list if len(i)>2]
print(len(text_list))


abstract_list = []
for txt in text_list:
    pmid, dic = extract_dic(txt)
    abstract_list.append(dic)
    assert pmid in empty
    

with open(f'/home/gy237/project/Biomedical_datasets/total_pubmed/sample_200/sample_292_abstract.json', 'r', encoding='utf-8') as f:
    old_abstract = json.load(f)
abstract_list = old_abstract + abstract_list
print(len(abstract_list))
with open(f'/home/gy237/project/Biomedical_datasets/total_pubmed/sample_200/sample_300_abstract.json', 'w', encoding='utf-8') as f:
    json.dump(abstract_list, f, ensure_ascii=False, indent=4)

8
300
